In [1]:
import csv
import pandas as pd
import random
from ast import literal_eval
from itertools import chain
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from nltk.corpus import stopwords
import re
from collections import OrderedDict
import inflect
import more_itertools as mit
import textstat

p = inflect.engine()

wnl = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

In [47]:
Toxical = pd.read_csv("tsd_trial.csv")
SwearWords = pd.read_csv("SwearWords.csv")
SwearWordsList = SwearWords['SWEAR'].to_list() #.replace("-$", "", regex=True)
replacements=[('\xa0-', ''), ('\\\\', '')] 
for pat, repl in replacements:
    SwearWordsList1 = list(OrderedDict.fromkeys([stemmer.stem(wnl.lemmatize(re.sub(pat, repl, i.lower()))) for i in SwearWordsList] + [re.sub(pat, repl, i.lower()) for i in SwearWordsList]))
# SwearWordsList2 = list(OrderedDict.fromkeys([p.plural(word.lower()) for word in SwearWordsList1] + [p.singular(word.lower()) for word in SwearWordsList1]))
SwearWordsList = [w for w in SwearWordsList1 if not w in stop_words]
Toxical.head()

,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."
3,"[87, 88, 89, 90, 91, 92]",Trump Claimed that Russia will never invade th...
4,[],As long as your willing to pay a lot more for ...


In [48]:
# def checkToxicalWords (sampledText, index):
#     toxicalWords = []
#     charList = list(sampledText[int(i)] for i in literal_eval(Toxical.iloc[index][0])) #Extract character from there
#     print(charList)
#     if ' ' in charList:
#         splitList = [i for i,x in enumerate(charList) if x == ' '] #Get the index of " " as splitter list
#         temp = zip(chain([0], splitList), chain(splitList, [None])) #Split the list by " "
#         res = list(''.join(charList[i : j]) for i, j in temp)
#         print(res)
#         if len(res) > 0:
#             resMod = [res[0]] + [i[1:] for i in res[1:]] #Remove the starting " " for the words
# #                 print(resMod) 
#         toxicalWords = resMod
#     else:
#         toxicalWords = [str(''.join(charList))]
# #     toxicalWords =  [i for i in toxicalWords] 
#     return toxicalWords

def checkToxicalWords (sampledText, index, dataframe):
    toxicalWords = []
    sepCharList = [list(group) for group in mit.consecutive_groups(literal_eval(dataframe.iloc[index][0]))]
    charList = []
    for word in sepCharList:
        charList.append(list(sampledText[int(i)] for i in word)) #Extract character from there
    spanList = [None] * len(charList)
    for i in range(len(charList)):
        spanList[i] = str(''.join(charList[i]))
    return spanList

In [130]:
sampledRow = random.randint(0, len(Toxical) - 1) #Sample a random row
# sampledRow = 0
print(sampledRow)
sampledText = Toxical.iloc[sampledRow][1] #Extract document text from that row

checkToxicalWords (sampledText, sampledRow, Toxical)

443


['Trump', 'buffoon', 'disgusting']

In [49]:
# Toxical['textSplit'] = [i.split() for i in Toxical['text']]
# sampledRow = random.randint(0, len(Toxical) - 1) #Sample a random row
# sampledText = Toxical.iloc[sampledRow][1] #Extract document text from that row
def getSwearIndex (sampledText, index, dataFrame):
    indexList = []
#     sampledTextCleaned = [stemmer.stem(wnl.lemmatize(re.sub(pat, repl, i.lower()))) for i in sampledText]
    toxicSpan = []
    toxicSpanCleaned = []
    toxicSpanCleanedidx = []
    charCounts = textstat.char_count(sampledText, ignore_spaces = True)  ##Count char counts
    senSplit = sampledText.split()
    wordCounts = len(senSplit) ## Count word num
    for idx, val in enumerate(senSplit):
        curWord = re.sub('[ ,.\'\"\"?]*$', '', stemmer.stem(wnl.lemmatize(re.sub(pat, repl, val.lower())))) #Stemmer, Encoding to UTF-8, and replace symbols
#         print(curWord)
        stopwords = [w for w in curWord if w in stop_words] ## Count how many stopwords in the data
        if curWord in SwearWordsList:
#             print(curWord)
            toxicSpan.append(val)
            toxicSpanCleaned.append(curWord)
            toxicSpanCleanedidx.append(idx)
#     intersection = set(sampledTextCleaned.split()).intersection(SwearWordsList)
    toxicalWordsLabeled = checkToxicalWords(sampledText, index, dataFrame)
    if len(toxicSpanCleaned) > 0:
        for i in toxicSpan: 
            indexList.append([item for item in range(sampledText.find(i), sampledText.find(i) + len(i))])
        indexList = [val for sublist in indexList for val in sublist]
    return senSplit, toxicalWordsLabeled, toxicSpanCleaned, indexList, wordCounts, charCounts, len(stopwords)

def generateNewColumns (dataFrame):
    dataFrame['textSplit'] = ''
    dataFrame['labeled_toxical_words'] = ''
    dataFrame['candidate_toxical_words'] = ''
    dataFrame['candidate_span'] = ''
    dataFrame['wordCounts'] = ''
    dataFrame['charCounts'] = ''
    dataFrame['stopCounts'] = ''
    
    for index, row in dataFrame.iterrows():
        row['textSplit'], row['labeled_toxical_words'], row['candidate_toxical_words'], row['candidate_span'], row['wordCounts'], row['charCounts'], row['stopCounts'] = getSwearIndex(row['text'].lower(), index, dataFrame)
    return dataFrame

In [50]:
ToxicalMatched = generateNewColumns(Toxical)
ToxicalMatched

,spans,text,textSplit,labeled_toxical_words,candidate_toxical_words,candidate_span,wordCounts,charCounts,stopCounts
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...,"[because, he's, a, moron, and, a, bigot., it's...","[moron, bigot]","[moron, bigot]","[15, 16, 17, 18, 19, 27, 28, 29, 30, 31, 32]",14,61,3
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...,"[how, about, we, stop, protecting, idiots, and...",[idiots],[idiot],"[29, 30, 31, 32, 33, 34]",26,115,4
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th...","[if, people, were, smart,, they, would, boycot...",[idiots],[idiot],"[166, 167, 168, 169, 170, 171]",29,136,2
3,"[87, 88, 89, 90, 91, 92]",Trump Claimed that Russia will never invade th...,"[trump, claimed, that, russia, will, never, in...",[stupid],[stupid],"[87, 88, 89, 90, 91, 92]",19,90,0
4,[],As long as your willing to pay a lot more for ...,"[as, long, as, your, willing, to, pay, a, lot,...",[],[hypocrit],"[188, 189, 190, 191, 192, 193, 194, 195, 196, ...",42,169,1
...,...,...,...,...,...,...,...,...,...
685,"[129, 130, 131, 132, 133, 134]",But ... Trump's not bluffing. He's prepared to...,"[but, ..., trump's, not, bluffing., he's, prep...",[stupid],[stupid],"[129, 130, 131, 132, 133, 134, 135]",30,136,0
686,"[126, 127, 128, 129, 130, 131]",Can't believe the limited knowledge of this Ar...,"[can't, believe, the, limited, knowledge, of, ...",[stupid],[stupid],"[126, 127, 128, 129, 130, 131, 132]",21,111,4
687,"[24, 25, 26, 27, 28, 29]",I think it conservative idiots who cannot reac...,"[i, think, it, conservative, idiots, who, cann...",[idiots],[idiot],"[24, 25, 26, 27, 28, 29]",10,53,5
688,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",You're an id*ot...Go away.,"[you're, an, id*ot...go, away.]",[you're an id*ot],[],[],4,23,3


In [387]:
ToxicalMatched[["candidate_span"]].to_csv("originalSubmission.txt", sep='\t')

# Training_Trail Data Set

In [51]:
trailData = pd.read_csv("tsd_trial.csv")
trainingData = pd.read_csv("tsd_train.csv")
dataFrames = [trailData, trainingData]
toxicDF = pd.concat(dataFrames).reset_index(drop = True)

In [52]:
print("The shape of data is %s.\n" % str(toxicDF.shape))
print("Null values:")
print(toxicDF.isnull().sum())
print("\n")
toxicDF.head()

The shape of data is (8629, 2).

Null values:
spans    0
text     0
dtype: int64




,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."
3,"[87, 88, 89, 90, 91, 92]",Trump Claimed that Russia will never invade th...
4,[],As long as your willing to pay a lot more for ...


In [5]:
## Creation of SwearWords List
SwearWordsList = SwearWords['SWEAR'].to_list() #.replace("-$", "", regex=True)
replacements=[('\xa0-', ''), ('\\\\', '')] 
for pat, repl in replacements:
    SwearWordsList1 = list(OrderedDict.fromkeys([stemmer.stem(wnl.lemmatize(re.sub(pat, repl, i.lower()))) for i in SwearWordsList] + [re.sub(pat, repl, i.lower()) for i in SwearWordsList]))
SwearWordsList = [w for w in SwearWordsList1 if not w in stop_words]

In [53]:
ToxicalMatched = generateNewColumns(toxicDF)

In [27]:
trailDF = toxicDF[:1000]

In [105]:
# for index, row in trailDF.iterrows():
#     row['charCount'] = textstat.char_count(row['text'], ignore_spaces = True)
#     row['wordCount'] = len(row['text'].split())
# #     for lineNo, line in enumerate(row['text']):
# #         print(line.split())

In [28]:
ToxicalMatched = generateNewColumns(trailDF)

C:\Users\Hamilton\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Hamilton\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Hamilton\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [55]:
ToxicalMatched.iloc[4876 : 4896, : ]

,spans,text,textSplit,labeled_toxical_words,candidate_toxical_words,candidate_span,wordCounts,charCounts,stopCounts
4876,"[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2...",Lots of so-called Christian suckers. The same ...,"[lots, of, so-called, christian, suckers., the...","[christian suckers, lies, fools, fool]",[],[],40,183,3
4877,"[157, 158, 159, 160]","But, but, Trudeau is a feminists, nand his ca...","[but,, but,, trudeau, is, a, feminists,, nand,...",[fool],[],[],31,132,2
4878,[],Moron-eau is in over his head. Home Capital i...,"[moron-eau, is, in, over, his, head., home, ca...",[],[idiot],"[132, 133, 134, 135, 136]",28,124,0
4879,"[362, 363, 364, 365, 366, 367, 368, 369, 370, ...","In one of my career choices, I had to make reg...","[in, one, of, my, career, choices,, i, had, to...","[liars, damned liars]",[],[],63,337,9
4880,"[44, 45, 46, 47, 48, 49, 67, 68, 69, 70, 71]","Perhaps we need a referendum take classify ""st...","[perhaps, we, need, a, referendum, take, class...","[stupid, class]",[],[],22,115,4
4881,"[15, 16, 17, 18]",Passing on the crap cards that were dealt you ...,"[passing, on, the, crap, cards, that, were, de...",[crap],[],[],32,159,2
4882,"[853, 854, 855, 856]",The treatment of the Rohingya Muslims in Myanm...,"[the, treatment, of, the, rohingya, muslims, i...",[rape],[],[],165,794,3
4883,"[84, 85, 86, 87, 88, 89]",What is it with women that they will tolerate ...,"[what, is, it, with, women, that, they, will, ...",[stupid],[stupid],"[84, 85, 86, 87, 88, 89]",27,113,2
4884,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","Tell me, are there other non-contributors that...","[tell, me,, are, there, other, non-contributor...","[tell me, are there other non-contributors tha...",[],[],101,546,1
4885,"[19, 20, 21, 22, 23, 24, 25, 26, 27]",LOL...love it when stupidity exposes itself on...,"[lol...love, it, when, stupidity, exposes, its...",[stupidity],[stupid],"[19, 20, 21, 22, 23, 24, 25, 26, 27]",9,52,3


In [38]:
ToxicalMatched.iloc[ : , -3 : ].mean(axis = 0)

wordCounts     34.440
charCounts    162.503
stopCounts      2.906
dtype: float64

In [40]:
ToxicalMatched.iloc[ : , -3 : ].std(axis = 0)

wordCounts     33.800812
charCounts    159.213199
stopCounts      2.160980
dtype: float64

In [56]:
wordLimit = ToxicalMatched.iloc[ : , -3 : ].mean(axis = 0)[0] + ToxicalMatched.iloc[ : , -3 : ].std(axis = 0)[0]
int(wordLimit)

70

In [3]:
RC_2018_01_Text_Combined = pd.read_csv("RC_2018_01_Text_Combined.csv")
RC_2018_01_Text_Combined.head()

C:\Users\Hamilton\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,id,Perpective_Score,toxicity,body
0,0,ds0n31r,0.897231,1,I personally just don\u2019t like the way it l...
1,1,ds0n31v,0.044559,0,NaN
2,2,ds0n31v,0.044559,0,NaN
3,14,ds0n31w,0.084523,0,NaN
4,27,ds0n31x,0.214329,0,NaN


In [4]:
RC_2018_01_Text_Combined_Not_Null = RC_2018_01_Text_Combined[RC_2018_01_Text_Combined.body.notnull()]

In [10]:
RC_2018_01_Text_Combined.head(10)

,Unnamed: 0,id,Perpective_Score,toxicity,body
0,0,ds0n31r,0.897231,1,I personally just don\u2019t like the way it l...
1,1,ds0n31v,0.044559,0,NaN
2,2,ds0n31v,0.044559,0,NaN
3,14,ds0n31w,0.084523,0,NaN
4,27,ds0n31x,0.214329,0,NaN
5,37,ds0n31x,0.214329,0,NaN
6,40,ds0n31y,0.715958,1,NaN
7,53,ds0n31z,0.280021,0,NaN
8,55,ds0n31z,0.280021,0,NaN
9,66,ds0n320,0.178349,0,NaN
